In [84]:
'''
# 이 부분은 처음 한번만 실행하면 됌.
# 코드 수정 - "The reason is that the last Ubuntu update update supports chromium driver just via snap."
# 최근 우분투 업데이트에서 크롬 드라이버 설치를 snap을 이용해서만 하도록 바뀜
# 고로 snap 없이 설치하는 아래 우회 코드로 변경
# 출처 : https://colab.research.google.com/drive/1cbEvuZOhkouYLda3RqiwtbM-o9hxGLyC
# 출처2 : https://stackoverflow.com/questions/75155063/selenium-use-chrome-on-colab-got-unexpectedly-exited

%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium
'''

'\n# 이 부분은 처음 한번만 실행하면 됌.\n# 코드 수정 - "The reason is that the last Ubuntu update update supports chromium driver just via snap."\n# 최근 우분투 업데이트에서 크롬 드라이버 설치를 snap을 이용해서만 하도록 바뀜\n# 고로 snap 없이 설치하는 아래 우회 코드로 변경\n# 출처 : https://colab.research.google.com/drive/1cbEvuZOhkouYLda3RqiwtbM-o9hxGLyC\n# 출처2 : https://stackoverflow.com/questions/75155063/selenium-use-chrome-on-colab-got-unexpectedly-exited\n\n%%shell\n# Ubuntu no longer distributes chromium-browser outside of snap\n#\n# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap\n\n# Add debian buster\ncat > /etc/apt/sources.list.d/debian.list <<\'EOF\'\ndeb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main\ndeb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main\ndeb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster

In [85]:
#필요한 것 import
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd

In [86]:
#input값 설정
user_input = input("기준금리와 CD금리, 그리고 예상하는 CD금리-기준금리를 입력하세요(ex : 3.5/3.75/0.15)" )
user_input = user_input.split("/")

기준금리와 CD금리, 그리고 예상하는 CD금리-기준금리를 입력하세요(ex : 3.5/3.75/0.15)3.5/3.75/0.15


In [87]:
#설정

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

In [88]:
#url 지정
url = 'http://www.smbs.biz/Exchange/IRS.jsp'
driver.get(url)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [89]:
# 소스 가져오기
table = soup.find('table', summary='기일물/1Y/2Y/3Y/4Y/5Y/7Y/10Y')

In [122]:
#가져오는 코드

irs_table = table.find_all('tbody')
data = []

for element in irs_table:
    selected_text = element.get_text().strip().split('\n\n')
    selected_text = [x.split('\n') for x in selected_text]
    # 빈 리스트인 경우 data에 추가하지 않음
    if selected_text != " ":
        data.extend(selected_text)

df = pd.DataFrame(data)

In [123]:
# 필요한 데이터만 가공

df.loc[0, [1, 2, 3, 4]] = df.iloc[0, [0, 1, 2, 3]].values
df = df.drop(columns=[0 ,1, 2, 3]).drop(index=df.index[5:])
df.index = ['0.5','0.75','1.00','1.50','2.00']
df.columns = ["Spot"]
df = df.astype(float)

In [124]:
df

,Spot
0.5,3.7975
0.75,3.8100
1.00,3.7900
1.50,3.7300
2.00,3.6625


In [129]:
#생성
df.loc['0.25'] = user_input[1]
df.loc['1.25'] = (df.loc['1.00'] + df.loc['1.50'])/2
df.loc['1.75'] = (df.loc['1.50'] + df.loc['2.00'])/2
df = df.sort_index()
df = df.astype(float)

In [130]:
df['DF'] = None
df['Zero']=None
df['Forward']=None

In [131]:
df

,Spot,DF,Zero,Forward
0.25,3.75000,None,None,None
0.5,3.79750,None,None,None
0.75,3.81000,None,None,None
1.00,3.79000,None,None,None
1.25,3.76000,None,None,None
1.50,3.73000,None,None,None
1.75,3.69625,None,None,None
2.00,3.66250,None,None,None


In [136]:
df.loc['0.25','DF'] = 1/(1+df.loc['0.25','Spot']/100*0.25) #맨 앞에거는 해야함

#for 문으로 돌려서

In [137]:
df

,Spot,DF,Zero,Forward
0.25,3.75000,0.990712,None,None
0.5,3.79750,None,None,None
0.75,3.81000,None,None,None
1.00,3.79000,None,None,None
1.25,3.76000,None,None,None
1.50,3.73000,None,None,None
1.75,3.69625,None,None,None
2.00,3.66250,None,None,None
